In [1]:
#first part:
#a test for how many score can a "perfect model" get after picture was resized
#image resize to IMG_SIZE ,then resize back to (240,428)
#compute fm-score using original picture and resized picture

#second part:
#noticed that py_sod_metric do normalization on your prediction by default,
#so i wanna test whether the same thing was done on leaderboard.

In [1]:
import os
import cv2
from torchvision.io import read_image, ImageReadMode, write_png
import torchvision.transforms.v2 as v2
import torch.nn.functional as F
import py_sod_metrics

# **about model output size**

In [3]:
from model import *
from torchsummary import summary
model = UNet((240,240))
#e = VAEencoder_res(10)
#d = VAEdecoder_res(10)
#model = VAE(decoder=d, encoder=e)
model.to("cuda:4")
summary(model, (3,240,240))
torch.jit.script(model).save('weights/test.pt')

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [3]:
models = []
for _ in range(1000):
    models.append(UNet((240,240)))
    models[-1].to('cuda:7')
    if _ % 20 ==0:
        print(1)

NameError: name 'UNet' is not defined

# **about resize**

In [2]:
path = './Training_dataset/label_img/TRA_RI_2000000.png'
IMG_SIZE = (240, 240)

In [5]:
def score(pred, gt):
    FMv2 = py_sod_metrics.FmeasureV2(
        metric_handlers={
            "fm": py_sod_metrics.FmeasureHandler(with_dynamic=True, with_adaptive=False, beta=0.3),
        }
    )
    FMv2.step(pred=pred, gt=gt)
    fmv2 = FMv2.get_results()
    print("mean F score: ",fmv2["fm"]["dynamic"].mean())
    return fmv2["fm"]["dynamic"].mean()

In [7]:
def test(img_path):
    img = read_image(img_path, mode = ImageReadMode.GRAY)
    #img = ((img>128)*255).float()
    img1_1 = v2.Resize(IMG_SIZE, antialias = True)(img)
    img1_1 = v2.ToDtype(torch.float32, scale = True)(img1_1)
    #img1_1 = ((img1_1>0.5)*255.0)
    img1_2 = v2.Resize((240,428), antialias = True)(img1_1)
    img1_3 = F.interpolate(img1_1.unsqueeze(0), size=(240, 428), mode='bilinear', align_corners=False).squeeze(0)
    img1_4 = F.interpolate(img.unsqueeze(0), size=(240, 428), mode='bilinear', align_corners=False).squeeze(0)

    img1_2 = (img1_2>0.5)*255
    img1_3 = (img1_3>0.5)*255
    img2_0 = img.squeeze(0).numpy()
    img2_2 = img1_2.squeeze(0).numpy()
    img2_3 = img1_3.squeeze(0).numpy()
    img2_4 = img1_4.squeeze(0).numpy()


    print('using v2.Resize(antialias=True) for second resize')
    score(img2_2,img2_0)
    print('----')
    print("using F.interpolate(mode='bilinear', align_corners=False) for second resize")
    score(img2_3,img2_0)

    #img3_2 = img1_2.numpy()
    #print((img3_2==img2_2).all())
    #print(img.squeeze(0).numpy().shape, img2_2.shape)

    print('----')
    print('only resize once((240,428)->(240,428)) using F.interpolate(mode="bilinear", align_corners=False)')
    score(img2_4, img2_0)

    print('----')
    print('score between img itself')
    score(img.squeeze(0).numpy(),img.squeeze(0).numpy())
test(path)

using v2.Resize(antialias=True) for second resize
mean F score:  0.9957230709909372
----
using F.interpolate(mode='bilinear', align_corners=False) for second resize
mean F score:  0.9957230709909372
----
only resize once((240,428)->(240,428)) using F.interpolate(mode="bilinear", align_corners=False)
mean F score:  0.996176789561585
----
score between img itself
mean F score:  0.996176789561585


In [7]:
path = './Training_dataset/img/TRA_RI_2000000.jpg'
img = read_image(path, mode = ImageReadMode.RGB)
img = v2.Grayscale()(img)
write_png(img, './tmp/test.png')

# **about normalization before scoring(in server end)**

In [ ]:
'''
#names = os.listdir('./submit/submit-0514-1338/')
#for name in names:
#    img = read_image('./submit/submit-0514-1338/'+name, mode = ImageReadMode.GRAY)
#    img_ = img//2
#    write_png(img_, './submit/submit-0514-1338/'+name)
'''

In [ ]:
'''
#import shutil
#shutil.make_archive('./submit/submit-0514-1338-dark/', 'zip', './submit/submit-0514-1338/')
'''

In [ ]:
#result
#submit-0514-1338, pub score = 0.673835
#submit-0514-1338-dark, pub score = 0.674352
#divide by 2 but got a higher score, lol.